<a href="https://colab.research.google.com/github/naenumtou/deepLearning/blob/main/X_rayCOVIDClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Change directory to Google drive
%cd /content/drive/My Drive/Colab Notebooks/cnn_model

/content/drive/My Drive/Colab Notebooks/cnn_model


In [ ]:
#Import library for model training 
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
print(tf.__version__)

2.2.0


In [ ]:
#Checking GPU running if return '/device:GPU:0' means GPU is used
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
#Loading features and target
X = pickle.load(open('features.pk', 'rb'))
Y = pickle.load(open('targets.pk', 'rb'))

#Normalization for (0,1) with max value
X = X/X.max()

In [ ]:
#Design the Convolutional Neural Networks (CNN)
model = Sequential()

#Layer 1: Convolutional layer 1
model.add(Conv2D(256, (3, 3), input_shape = X.shape[1:]))
model.add(Activation('relu'))

#Layer 2: Pooling layer
model.add(MaxPooling2D(pool_size = (2, 2)))

#Layer 3: Convolutional layer 2
model.add(Conv2D(256, (3, 3))) #Don't need input shape
model.add(Activation('relu'))

#Layer 4: Pooling layer
model.add(MaxPooling2D(pool_size = (2, 2)))

#Layer 5: Flatten layer
model.add(Flatten()) #Convert 3D feature map to 1D feature vectors

#Layer 6: Dense layer (Hidden layer)
model.add(Dense(64))

#Layer 7: Adding dropout layer to overcome over-fitting
model.add(Dropout(0.5))

#Layer 8: Output layer
model.add(Dense(1)) #Only one answer either cat or dog
model.add(Activation('sigmoid')) #Using 'sigmoid' since (0,1) answer

#Complie the model
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

#Model overview
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 398, 398, 256)     2560      
_________________________________________________________________
activation (Activation)      (None, 398, 398, 256)     0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 199, 199, 256)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 197, 197, 256)     590080    
_________________________________________________________________
activation_1 (Activation)    (None, 197, 197, 256)     0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 98, 98, 256)       0         
_________________________________________________________________
flatten (Flatten)            (None, 2458624)           0

In [ ]:
#Adding early stopping process
early_stop = EarlyStopping(monitor = 'val_loss', patience = 2)

In [ ]:
#Fitting the CNN model
model.fit(X, Y, batch_size = 16, epochs = 20, validation_split = 0.3)

Epoch 1/20
81/81 [==============================] - 61s 750ms/step - loss: 2.0039 - accuracy: 0.9023 - val_loss: 0.3126 - val_accuracy: 0.9152
Epoch 2/20
81/81 [==============================] - 57s 698ms/step - loss: 0.1075 - accuracy: 0.9651 - val_loss: 0.1252 - val_accuracy: 0.9621
Epoch 3/20
81/81 [==============================] - 56s 697ms/step - loss: 0.0599 - accuracy: 0.9806 - val_loss: 0.1016 - val_accuracy: 0.9693
Epoch 4/20
81/81 [==============================] - 56s 696ms/step - loss: 0.0384 - accuracy: 0.9876 - val_loss: 0.1186 - val_accuracy: 0.9639
Epoch 5/20
81/81 [==============================] - 56s 697ms/step - loss: 0.0140 - accuracy: 0.9953 - val_loss: 0.1009 - val_accuracy: 0.9639
Epoch 6/20
81/81 [==============================] - 56s 697ms/step - loss: 0.0197 - accuracy: 0.9953 - val_loss: 0.1009 - val_accuracy: 0.9711
Epoch 7/20
81/81 [==============================] - 56s 697ms/step - loss: 0.2435 - accuracy: 0.9744 - val_loss: 0.4511 - val_accuracy: 0.9729

In [ ]:
#Save model
model.save('covid-19_cnn.model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: covid-19_cnn.model/assets


In [ ]:
#Import library for model prediction
import os
import cv2

In [ ]:
#Model prediction
results = ['normal', 'covid-19']
size = 400 #Same as training model

#Load trained model
model_pred = tf.keras.models.load_model('covid-19_cnn.model')

In [ ]:
for i in os.listdir(os.getcwd()):
  if i.endswith('.jpg'):
    img = cv2.imread(i)
    img = cv2.resize(img, (size, size))
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_array = img_gray.reshape(-1, size, size, 1)
    res = model_pred(img_array)
    if i[:-7] == results[int(res[0][0])]:
      print('True')
    else:
      print('False')
    print(f'Actual X-Ray film is {i[:-7]}')
    print(f'Model result is {results[int(res[0][0])]}')
    cv2.putText(img, str(results[int(res[0][0])]), (size - 200, size - 200), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 0, 0), 2)
    cv2.imwrite(f'result_{i}', img)

True
Actual X-Ray film is normal
Model result is normal
True
Actual X-Ray film is normal
Model result is normal
False
Actual X-Ray film is normal
Model result is covid-19
True
Actual X-Ray film is normal
Model result is normal
True
Actual X-Ray film is normal
Model result is normal
True
Actual X-Ray film is covid-19
Model result is covid-19
True
Actual X-Ray film is covid-19
Model result is covid-19
True
Actual X-Ray film is covid-19
Model result is covid-19
False
Actual X-Ray film is result_normal
Model result is normal
